In [33]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from imblearn.over_sampling import SMOTE

from sklearn.feature_selection import SelectFromModel

from xgboost import XGBClassifier

import xgboost as xgb


In [2]:
df=pd.read_csv('data.csv', sep=';')

In [3]:
df

,Unnamed: 0,Enc_rph,REGION,SEXO,TRAMO_ETARIO,NACIONALIDAD,CINE_EME_RED,EST_CONYUGAL,SUBMUESTRA,proveedor,...,motivacion,gasto_anual,ingreso_anual,ganancia_final_men,tramos_ganancias,sum_l3_t,sum_l4_t,trabajo_no_remunerado,trabajo_remunerado,trabajo_global
0,1,5930,6,2,6,0,3,1,3,1,...,2.0,11324000,18060000,5e+05,2.0,"6,66666666666667",0,"6,66666666666667",8,"14,6666666666667"
1,2,1052,9,1,6,0,2,1,2,2,...,2.0,1224000,3600000,2e+05,1.0,0,0,0,12,12
2,3,4203,4,1,2,0,2,2,2,2,...,3.0,124880000,166879992,3500000,6.0,"1,25",1,"2,25",8,"10,25"
3,4,7738,5,2,2,0,4,2,2,1,...,2.0,2544000,6192000,250000,1.0,4,3,7,4,11
4,5,6539,11,1,6,0,2,1,2,2,...,3.0,0,3500000,7e+05,2.0,0,0,0,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6937,6938,2310,10,1,4,0,4,1,3,2,...,3.0,5718000,10800006,5e+05,2.0,"0,416666666666667",0,"0,416666666666667",8,"8,41666666666667"
6938,6939,7104,8,2,2,0,5,3,3,1,...,3.0,645000,1935000,520000,2.0,2,0,2,10,12
6939,6940,8933,10,1,3,0,2,3,3,2,...,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN
6940,6941,3322,13,2,4,0,4,5,3,2,...,3.0,3800000,6200000,230000,1.0,"2,25",0,"2,25",4,"6,25"


In [4]:
df.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6942 entries, 0 to 6941
Data columns (total 426 columns):
 #    Column                      Dtype  
---   ------                      -----  
 0    Unnamed: 0                  int64  
 1    Enc_rph                     int64  
 2    REGION                      int64  
 3    SEXO                        int64  
 4    TRAMO_ETARIO                int64  
 5    NACIONALIDAD                int64  
 6    CINE_EME_RED                int64  
 7    EST_CONYUGAL                int64  
 8    SUBMUESTRA                  int64  
 9    proveedor                   int64  
 10   a1                          int64  
 11   a2                          float64
 12   a3                          float64
 13   a3_otro                     float64
 14   a4                          float64
 15   a5                          float64
 16   a5_otro                     float64
 17   a6                          float64
 18   a7                          float64
 19   a8  

In [5]:
variables=['SEXO','TRAMO_ETARIO','EST_CONYUGAL'	, 'CINE_EME_RED','c1_caenes_1d_red'	,'b3','b8','REGION'	,'g10_1','g10_2','e3','b5_1','b5_2','e8_1', 'e8_2',	'e8_3','k12_1', 'j1']

In [6]:
#seleccionamos las variables que nos interesan
df1=df[variables]


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6942 entries, 0 to 6941
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SEXO              6942 non-null   int64  
 1   TRAMO_ETARIO      6942 non-null   int64  
 2   EST_CONYUGAL      6942 non-null   int64  
 3   CINE_EME_RED      6942 non-null   int64  
 4   c1_caenes_1d_red  6579 non-null   float64
 5   b3                6579 non-null   float64
 6   b8                6579 non-null   float64
 7   REGION            6942 non-null   int64  
 8   g10_1             6579 non-null   float64
 9   g10_2             6579 non-null   float64
 10  e3                6579 non-null   float64
 11  b5_1              6579 non-null   float64
 12  b5_2              6579 non-null   float64
 13  e8_1              6579 non-null   float64
 14  e8_2              6579 non-null   float64
 15  e8_3              6579 non-null   float64
 16  k12_1             6579 non-null   float64


In [8]:
#elimina filas con valor 96 en columna e3
df1 = df1[df1.e3 != 96]


In [9]:
#elimina filas con nan en la columna c1_caenes_1d_red
df1.dropna(subset=['c1_caenes_1d_red'], inplace=True)

In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6531 entries, 0 to 6941
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SEXO              6531 non-null   int64  
 1   TRAMO_ETARIO      6531 non-null   int64  
 2   EST_CONYUGAL      6531 non-null   int64  
 3   CINE_EME_RED      6531 non-null   int64  
 4   c1_caenes_1d_red  6531 non-null   float64
 5   b3                6531 non-null   float64
 6   b8                6531 non-null   float64
 7   REGION            6531 non-null   int64  
 8   g10_1             6531 non-null   float64
 9   g10_2             6531 non-null   float64
 10  e3                6531 non-null   float64
 11  b5_1              6531 non-null   float64
 12  b5_2              6531 non-null   float64
 13  e8_1              6531 non-null   float64
 14  e8_2              6531 non-null   float64
 15  e8_3              6531 non-null   float64
 16  k12_1             6531 non-null   float64
 17  

In [11]:
df1=df1.astype('int')

In [12]:
#convierte variables a categoricas
df1=df1.astype('category')

In [13]:
df1.info(10)

<class 'pandas.core.frame.DataFrame'>
Index: 6531 entries, 0 to 6941
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   SEXO              6531 non-null   category
 1   TRAMO_ETARIO      6531 non-null   category
 2   EST_CONYUGAL      6531 non-null   category
 3   CINE_EME_RED      6531 non-null   category
 4   c1_caenes_1d_red  6531 non-null   category
 5   b3                6531 non-null   category
 6   b8                6531 non-null   category
 7   REGION            6531 non-null   category
 8   g10_1             6531 non-null   category
 9   g10_2             6531 non-null   category
 10  e3                6531 non-null   category
 11  b5_1              6531 non-null   category
 12  b5_2              6531 non-null   category
 13  e8_1              6531 non-null   category
 14  e8_2              6531 non-null   category
 15  e8_3              6531 non-null   category
 16  k12_1             6531 non-nu

In [14]:
df1.describe(include='all')

,SEXO,TRAMO_ETARIO,EST_CONYUGAL,CINE_EME_RED,c1_caenes_1d_red,b3,b8,REGION,g10_1,g10_2,e3,b5_1,b5_2,e8_1,e8_2,e8_3,k12_1,j1
count,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531
unique,2,6,7,6,8,5,4,16,14,14,6,11,10,2,2,2,3,5
top,1,5,1,3,5,1,1,13,2,13,5,1,10,2,2,2,2,4
freq,3693,1651,2384,3072,1851,2488,3011,1183,4670,3746,3924,4717,5919,5784,6093,5807,3439,4980


In [15]:
#en columna SEXO, remmplaza  2 por 0
df1['SEXO'].replace(2, 0, inplace=True)


In [ ]:
#reemplaza 2 por 0 en columnas e8_1, e8_2, e8_3
df1['e8_1'].replace(2, 0, inplace=True)
df1['e8_2'].replace(2, 0, inplace=True)
df1['e8_3'].replace(2, 0, inplace=True)

In [17]:
df1.describe(include='all')

,SEXO,TRAMO_ETARIO,EST_CONYUGAL,CINE_EME_RED,c1_caenes_1d_red,b3,b8,REGION,g10_1,g10_2,e3,b5_1,b5_2,e8_1,e8_2,e8_3,k12_1,j1
count,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531,6531
unique,2,6,7,6,8,5,4,16,14,14,6,11,10,2,2,2,3,5
top,1,5,1,3,5,1,1,13,2,13,5,1,10,0,0,0,2,4
freq,3693,1651,2384,3072,1851,2488,3011,1183,4670,3746,3924,4717,5919,5784,6093,5807,3439,4980


In [ ]:
#separa las variables predictoras de la variable objetivo, la variable objetivo es e3

X = df1.drop('e3', axis=1)
y = df1['e3']

In [19]:
var=['TRAMO_ETARIO','EST_CONYUGAL', 'CINE_EME_RED','c1_caenes_1d_red','b3','b8','REGION','g10_1','g10_2','b5_1','b5_2','k12_1', 'j1']

In [ ]:
#en X crea variables que estan en lista var  a dummy
X = pd.get_dummies(X, columns=var)

In [ ]:
#X a boleano
X = X.astype('bool')

In [22]:
X.info(10)

<class 'pandas.core.frame.DataFrame'>
Index: 6531 entries, 0 to 6941
Data columns (total 113 columns):
 #    Column                Dtype
---   ------                -----
 0    SEXO                  bool 
 1    e8_1                  bool 
 2    e8_2                  bool 
 3    e8_3                  bool 
 4    TRAMO_ETARIO_1        bool 
 5    TRAMO_ETARIO_2        bool 
 6    TRAMO_ETARIO_3        bool 
 7    TRAMO_ETARIO_4        bool 
 8    TRAMO_ETARIO_5        bool 
 9    TRAMO_ETARIO_6        bool 
 10   EST_CONYUGAL_1        bool 
 11   EST_CONYUGAL_2        bool 
 12   EST_CONYUGAL_3        bool 
 13   EST_CONYUGAL_4        bool 
 14   EST_CONYUGAL_5        bool 
 15   EST_CONYUGAL_6        bool 
 16   EST_CONYUGAL_99       bool 
 17   CINE_EME_RED_1        bool 
 18   CINE_EME_RED_2        bool 
 19   CINE_EME_RED_3        bool 
 20   CINE_EME_RED_4        bool 
 21   CINE_EME_RED_5        bool 
 22   CINE_EME_RED_6        bool 
 23   c1_caenes_1d_red_1    bool 
 24   c1_caen

In [23]:
y.value_counts()

e3
5    3924
2    1289
1     709
3     367
4     176
6      66
Name: count, dtype: int64

In [24]:
# X e y enteros
X = X.astype('int')
y = y.astype('int')


In [25]:
#en y reemplaza 1,2,3,4 por 1 restantes por 0
y.replace([1,2,3,4], 1, inplace=True)
y.replace([5,6], 0, inplace=True)

In [26]:
y.value_counts()

e3
0    3990
1    2541
Name: count, dtype: int64

In [27]:
#smote para balancear
smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()



e3
1    3990
0    3990
Name: count, dtype: int64

In [ ]:
#separa en train y test, 70% train, 30% test  train corresponde a los datos utilizados para entrenar el modelo y test a los datos utilizados para evaluar el modelo

X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.3, random_state=42)


## XGBoost como modelo de clasificación

In [34]:
model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=20, seed=123)

In [35]:
# seleccion de variables
selection = SelectFromModel(model, threshold=0.01)
select_X_train = selection.fit_transform(X_train, y_train)


# entrena el modelo con las variables seleccionadas
selection_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=20, seed=123)
selection_model.fit(select_X_train, y_train)


# evalua el modelo 
select_X_test = selection.transform(X_test)

y_pred = selection_model.predict(select_X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.74      0.76      1223
           1       0.74      0.78      0.76      1171

    accuracy                           0.76      2394
   macro avg       0.76      0.76      0.76      2394
weighted avg       0.76      0.76      0.76      2394

[[901 322]
 [262 909]]


In [1]:
(901+909)/(901+909+262+322)

0.7560568086883876

El modelo tiene una tasa de acierto (accuracy) del 76% = (901+909)/(901+909+262+322). los números en la diagonal de la matriz de confusión son los registros que el modelo clasifica correctamente.

Precisión para la categoria 0 es 0.77 =901/(901+262) y el recall 0.74=901/(901+322), de la misma forma lo pueden calcular para la categoría 1. Estos resultados los deben interpretar en la tesis.


In [ ]:
#variables mas importantes
importances = selection_model.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(select_X_train.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, selected_feat[indices[f]], importances[indices[f]]))
    
    
# con esto pueden ver las variables mas importantes e interpretar con sus palabras. Por qué creen que esas variables son las más importantes?    Esto lo hacen solo para algunas variables, por ejemplo 5

1. feature CINE_EME_RED_2 (0.078670)
2. feature g10_1_96 (0.073683)
3. feature b3_3 (0.066804)
4. feature CINE_EME_RED_1 (0.057046)
5. feature e8_3 (0.055222)
6. feature CINE_EME_RED_4 (0.048973)
7. feature CINE_EME_RED_3 (0.045745)
8. feature b3_88 (0.043515)
9. feature TRAMO_ETARIO_1 (0.040156)
10. feature g10_2_3 (0.037549)
11. feature CINE_EME_RED_5 (0.032373)
12. feature b5_1_9 (0.030517)
13. feature REGION_9 (0.029633)
14. feature c1_caenes_1d_red_4 (0.028102)
15. feature TRAMO_ETARIO_2 (0.027822)
16. feature EST_CONYUGAL_5 (0.025809)
17. feature b5_1_6 (0.025018)
18. feature b3_1 (0.024494)
19. feature c1_caenes_1d_red_3 (0.023742)
20. feature c1_caenes_1d_red_1 (0.021459)
21. feature EST_CONYUGAL_3 (0.020367)
22. feature REGION_2 (0.018796)
23. feature b5_1_8 (0.018745)
24. feature REGION_14 (0.018409)
25. feature EST_CONYUGAL_6 (0.016984)
26. feature EST_CONYUGAL_2 (0.016798)
27. feature b3_2 (0.014737)
28. feature c1_caenes_1d_red_5 (0.014423)
29. feature TRAMO_ETARIO_3 (0.01